## 1. 데이터 로드

In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Komoran

In [2]:
purpose = pd.read_csv("../data/수정데이터/용도별 목적대화 데이터.csv")
topic = pd.read_csv("../data/수정데이터/주제별 텍스트 일상 대화 데이터.csv")
sense = pd.read_csv("../data/수정데이터/일반상식 데이터.csv")
travelinfo = pd.read_csv("../data/수정데이터/여행정보 데이터.csv")
recommend = pd.read_csv("../data/수정데이터/여행지추천 데이터.csv")

purpose.dropna(inplace=True)
topic.dropna(inplace=True)
sense.dropna(inplace=True)
travelinfo.dropna(inplace=True)
recommend.dropna(inplace=True)

In [3]:
print(f'purpose shape: {purpose.shape}')
print(f'topic shape: {topic.shape}')
print(f'sense shape: {sense.shape}')
print(f'info shape: {travelinfo.shape}')
print(f'recommend shape: {recommend.shape}')

purpose shape: (225974, 1)
topic shape: (870138, 1)
sense shape: (68044, 3)
info shape: (11375, 4)
recommend shape: (980, 2)


In [4]:
print(purpose.columns)
print(topic.columns)
print(sense.columns)
print(travelinfo.columns)
print(recommend.columns)

Index(['text'], dtype='object')
Index(['text'], dtype='object')
Index(['intent', 'question', 'answer'], dtype='object')
Index(['city', 'sigungu', 'title', 'label'], dtype='object')
Index(['question', 'answer'], dtype='object')


In [5]:
total_data = list(purpose['text']) + list(topic['text']) + list(sense['question']) + list(sense['answer']) + \
            list(travelinfo['city']) + list(travelinfo['sigungu']) + list(travelinfo['title']) + list(recommend['question']) + list(recommend['answer'])

In [6]:
len(total_data)

1268285

In [7]:
df = pd.DataFrame({'text': total_data})
df['text'] = df['text'].str.strip()
df.to_csv('../data/수정데이터/통합데이터.csv', index=False)

## 2. 의도 분류 데이터 생성
* 0: 추천
* 1: 예약
* 2: 정보
* 3: 기타
* 4: nan

In [30]:
df = pd.read_csv('../data/수정데이터/save.csv')
total_data = df[['checked']]

In [31]:
total_data.shape

(1257779, 1)

In [32]:
total_data.isnull().sum()

checked    426
dtype: int64

In [34]:
total_data = total_data.dropna().reset_index(drop=True)

In [44]:
city = travelinfo['city'].unique().tolist() + travelinfo['sigungu'].str[:-1].unique().tolist()

In [45]:
city += ['전남', '경남', '충북', '전북', '경북', '충남', '세종']

In [46]:
reco = []
apo = []
info = []
etc = []
nan = []

In [49]:
for data in total_data['checked']:
    reco_keywords = ['곳', '어디', '결혼', '여행', '추천', '맛집', '관광', '여행지', '지역', '국내', '갈만'] + city
    apo_keywords = ['이름', '고객님', '시간', '예약', '본인', '입금', '렌트', '숙소', '호텔', '모텔', '오두막', '일정', '체크인', '숙박', '환불', '펜션', '캠핑', '풀빌라']
    info_keywords = ['상품', '전화', '번호', '연락', '안내', '문의', '방법', '주소', '접수', '전번', '정보', '축제', '행사', '놀', '놀거리', '박람회', '아쿠아리움', '명소']
    try:
        if any(keyword in data for keyword in reco_keywords):
            reco.append(data)
        elif any(keyword in data for keyword in apo_keywords):
            apo.append(data)
        elif any(keyword in data for keyword in info_keywords):
            info.append(data)
        else:
            etc.append(data)
    except:
        nan.append(data)

In [50]:
reco_df = pd.DataFrame({'text': reco})
reco_df['label'] = 0
reco_df.head()

,text,label
0,안내해 주신 방법을 시도해 봤는데 안 돼서요,0
1,휴대폰하고 신분증 챙기셔서 저희 서비스센터 쪽으로 방문하셔야 되세요 고객님,0
2,일단 고객님 고객님께서 설정하신 걸 입력하셨는데도 안 되신다면요,0
3,저희가 부산 해운대구에는 주소에 있는 센터가 있고요,0
4,저희 서비스센터로 오시면 됩니다 고객님,0


In [51]:
apo_df = pd.DataFrame({'text': apo})
apo_df['label'] = 1
apo_df.tail()

,text,label
61906,화진포 오토캠핑장,1
61907,황둔에 오토캠핑장,1
61908,휴가를 부탁해 캠핑장,1
61909,세계유교문화 축전 캠핑축제 2017,1
61910,오크밸리 캠핑 페스티벌 2017,1


In [52]:
info_df = pd.DataFrame({'text': info})
info_df['label'] = 2
info_df.tail()

,text,label
42338,DGIST 세계 명문 대학 조정 축제 2017,2
42339,DMZ 펀치볼 시래기축제 2018,2
42340,Hello! 과학마을축제 2017,2
42341,JDC 지구촌 문화축제 2017,2
42342,KF 청계천 음악 축제 2017,2


In [53]:
etc_df = pd.DataFrame({'text': etc})
etc_df['label'] = 3
etc_df.tail()

,text,label
794834,G-STAR G-LAND FESTIVAL 2017,3
794835,LG 드림페스티벌 2017,3
794836,Lifeplus 벚꽃 피크닉 페스티벌 2018,3
794837,SW 교육 페스티벌 2017,3
794838,What We Wear:왓 위 웨어 2017,3


In [54]:
nan

[]

In [55]:
total_train_data = pd.concat([reco_df, apo_df, info_df, etc_df], axis=0)
total_train_data.reset_index(drop=True, inplace=True)
total_train_data.shape

(1257354, 2)

In [56]:
total_train_data[total_train_data['label'] == 0]

,text,label
0,안내해 주신 방법을 시도해 봤는데 안 돼서요,0
1,휴대폰하고 신분증 챙기셔서 저희 서비스센터 쪽으로 방문하셔야 되세요 고객님,0
2,일단 고객님 고객님께서 설정하신 걸 입력하셨는데도 안 되신다면요,0
3,저희가 부산 해운대구에는 주소에 있는 센터가 있고요,0
4,저희 서비스센터로 오시면 됩니다 고객님,0
...,...,...
358256,강원도 가족여행지 곳 정리해뒀네요 장소 정해지면 네이버 블로그 리뷰로 꼭 정...,0
358257,저는 강원도에 안 번데기 다녀왔는데 너무 좋아서 추천드립니다 답변해 드리면서 강원...,0
358258,포항에 일본 가옥거리 추천해 드릴게요 작은 골목이라서 넓지는 않지만 옛날 생각나는...,0
358259,국제슬로시티로 지정된 전주 한옥마을 추천합니다 용산역에서 전라선 기차와 시외 고속...,0


In [57]:
total_train_data[total_train_data['label'] == 1]

,text,label
358261,반갑습니다 소속 상담사 이름입니다,1
358262,휴대폰 명의자 본인이요,1
358263,그건 고객님 설정하신 번호 눌러도 진입이 안 되세요 고객님,1
358264,조금 번거로우시겠지만 센터 쪽으로 내방해 주셔야 할 것 같으세요 고객님,1
358265,예 고객님 회현지구 쪽으로 확인해 드리겠습니다,1
...,...,...
420167,화진포 오토캠핑장,1
420168,황둔에 오토캠핑장,1
420169,휴가를 부탁해 캠핑장,1
420170,세계유교문화 축전 캠핑축제 2017,1


In [58]:
total_train_data[total_train_data['label'] == 2]

,text,label
420172,그러시다면 두 번째 방법은요,2
420173,아니시면 주소에 있는 작은 분점도 확인되고 있습니다,2
420174,주소 이 나을 거 같아요,2
420175,그러세요 그러시다면 주소 은 지하철 장산역 십사 번 출구에 있고요,2
420176,아까 주소 얘기하다가 끊어졌죠 주소에 있는 주소에요,2
...,...,...
462510,DGIST 세계 명문 대학 조정 축제 2017,2
462511,DMZ 펀치볼 시래기축제 2018,2
462512,Hello! 과학마을축제 2017,2
462513,JDC 지구촌 문화축제 2017,2


In [59]:
total_train_data[total_train_data['label'] == 3]

,text,label
462515,checked,3
462516,이게 왜 이렇게 된 거죠,3
462517,그거 내가 설정 안 해놨는데요,3
462518,그거 해봤지요 안 되네요,3
462519,어쩔 수 없지요 센터 가야 되겠네요,3
...,...,...
1257349,G-STAR G-LAND FESTIVAL 2017,3
1257350,LG 드림페스티벌 2017,3
1257351,Lifeplus 벚꽃 피크닉 페스티벌 2018,3
1257352,SW 교육 페스티벌 2017,3


In [60]:
total_train_data.to_csv('../data/수정데이터/total_train_data.csv', index=False)

## 3. 적절한 패딩 길이 탐색

In [61]:
total_train_data = pd.read_csv('../data/수정데이터/total_train_data.csv')

In [62]:
tokenizer = Komoran()

In [63]:
data_tokenized = [[token + '/' + POS for token, POS in tokenizer.pos(text)] for text in total_train_data['text']]

In [64]:
exclusion_tags = [
    "JKS", "JKC", "JKG", "JKO", "JKB", "JKV", "JKQ",
    "JX", "JC",
    "SF", "SP", "SS", "SE", "SO",
    "EP", "EF", "EC", "ETN", "ETM",
    "XSN", "XSV", "XSA",
]
def Exclusion_tags(x):
    return x in exclusion_tags

data_list = []
for i in range(len(data_tokenized)):
    temp = []
    for j in range(len(data_tokenized[i])):
        if Exclusion_tags(data_tokenized[i][j].split('/')[1]) is False:
            temp.append(data_tokenized[i][j].split('/')[0])
    data_list.append(temp)

In [65]:
len(data_list)

1257354

In [66]:
num_tokens = [len(sent) for sent in data_list]
num_tokens = np.array(num_tokens)
print('토큰 길이 평균:', num_tokens.mean())
print('토큰 길이 최대:', num_tokens.max())
print('토큰 길이 표준편차:', num_tokens.std())

토큰 길이 평균: 6.301614342500203
토큰 길이 최대: 155
토큰 길이 표준편차: 3.5888492470766002


In [67]:
MAX_SEQ_LEN = 25
def below_len_rate(max_len, data_list):
    cnt = 0
    for s in data_list:
        if (len(s) <= max_len):
            cnt = cnt + 1

    print(f'전체 샘플 중 길이가 {max_len} 이하인 샘플의 비율: {(cnt / len(data_list))}')
below_len_rate(MAX_SEQ_LEN, data_list)

전체 샘플 중 길이가 25 이하인 샘플의 비율: 0.9987624805742854


In [68]:
num_tokens.argmax()
data_tokenized[73475]

['남동생/NNP',
 '이랑/NNP',
 '너무/MAG',
 '데/NNB',
 '이/VCP',
 '면/EC',
 '데/NNB',
 '이/VCP',
 '면/EC',
 '하하/NNP']